# Part 1: Data scraping and cleaning

In [1]:
from bs4 import BeautifulSoup
import requests as r
import pandas as pd 

import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium # plotting library
import json

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans



In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
content = r.get(url).text
soup = BeautifulSoup(content, 'html.parser')
#print(soup.prettify())

In [3]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [4]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [5]:
df.shape

(103, 3)

# Part 2: Getting coordinates for postal codes

In [6]:
#import geocoder # import geocoder

In [7]:
''' initialize your variable to None
lat_lng_coords = None
latitude =[]
longitude =[]
count = 0
# loop until you get the coordinates
for postal_code in df.PostalCode:
    print(postal_code)
    count = 0
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        count += 1
        print(count)
        if count == 100:
            lat_lng_coords = list[0,0]
            break
    latitude.append(lat_lng_coords[0]) # Make list of latitudes
    longitude.append(lat_lng_coords[1]) # Make list of longitudes

#Merge latitude and longitude to dataframe
df['Latitude'] = latitude
df['Longitude'] = longitude

df.head()
'''

" initialize your variable to None\nlat_lng_coords = None\nlatitude =[]\nlongitude =[]\ncount = 0\n# loop until you get the coordinates\nfor postal_code in df.PostalCode:\n    print(postal_code)\n    count = 0\n    while(lat_lng_coords is None):\n        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))\n        lat_lng_coords = g.latlng\n        count += 1\n        print(count)\n        if count == 100:\n            lat_lng_coords = list[0,0]\n            break\n    latitude.append(lat_lng_coords[0]) # Make list of latitudes\n    longitude.append(lat_lng_coords[1]) # Make list of longitudes\n\n#Merge latitude and longitude to dataframe\ndf['Latitude'] = latitude\ndf['Longitude'] = longitude\n\ndf.head()\n"

In [8]:
coord = pd.read_csv("Geospatial_Coordinates.csv")
coord

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [9]:
df = df.merge(coord, left_on= 'PostalCode', right_on = 'Postal Code')
df.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",M6A,43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,M7A,43.662301,-79.389494


In [10]:
df.drop(columns=['Postal Code'], axis= 1, inplace= True)
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


# Part 3: Explore and Cluster Toronto.

In [39]:
#foursquare credentials
CLIENT_ID = '' # your Foursquare ID (Removed for privacy )
CLIENT_SECRET = '' # your Foursquare Secret
ACCESS_TOKEN = '' # your FourSquare Access Token
VERSION = '20180604'


In [12]:
# Create dataframe for neighborhoods in Toronto
df_toronto = df[df['Borough'].str.contains("Toronto")]
df_toronto.reset_index(inplace=True)
df_toronto.drop(columns='index', axis = 1, inplace=True)
df_toronto.head()

C:\Users\Sconzy\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [13]:
toronto_map = folium.Map(location=[43.654260,-79.360636], zoom_start = 13) #generate map centered around Regent Park and Harbourfront neighborhoods
# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
   [43.654260,-79.360636],
    radius=10,
    color='red',
    popup='Toronto',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(toronto_map)

# add the all neighborhoods as blue circle markers
for lat, lng, label in zip(df_toronto.Latitude, df_toronto.Longitude, df_toronto.Neighborhood):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(toronto_map)

toronto_map

### Exploring St. James Town neighborhood

In [14]:
neighborhood_latitude = df_toronto.loc[2, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_toronto.loc[2, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_toronto.loc[2, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of St. James Town are 43.6514939, -79.3754179.


### Getting the top 10 venues in St. James Town within a radius of 500 meters

In [40]:
LIMIT = 10
radius = 500
url_4s = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url_4s

'https://api.foursquare.com/v2/venues/explore?&client_id=&client_secret=&v=20180604&ll=43.6514939,-79.3754179&radius=500&limit=10'

In [16]:
#gettting the json result from Foursquare
results = r.get(url_4s).json()
results

{'meta': {'code': 200, 'requestId': '60f8e4bd4ffd057bcbad096a'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'St. Lawrence',
  'headerFullLocation': 'St. Lawrence, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 84,
  'suggestedBounds': {'ne': {'lat': 43.6559939045, 'lng': -79.36921018606671},
   'sw': {'lat': 43.646993895499996, 'lng': -79.3816256139333}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '574ad72238fa943556d93b8e',
       'name': 'Gyu-Kaku Japanese BBQ',
       'location': {'address': '81 Church St',
        'crossStreet': 'at Adelaide St E',
        'lat': 43.651422275497914,
        'lng': -79.37504693687086,
        'labeledLatLngs':

In [17]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Cleaning the json and placing it in dataframe

In [18]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-18-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Gyu-Kaku Japanese BBQ,Japanese Restaurant,43.651422,-79.375047
1,Terroni,Italian Restaurant,43.650927,-79.375602
2,GEORGE Restaurant,Restaurant,43.653346,-79.374445
3,Fahrenheit Coffee,Coffee Shop,43.652384,-79.372719
4,Crepe TO,Creperie,43.650063,-79.374587


# Exploring the rest of Toronto Neighborhoods

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = r.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
# get venues into new dataframe
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadi

In [21]:
print(toronto_venues.shape)
toronto_venues.head()

(344, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [22]:
#cheking  number of venues for each neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,10,10,10,10,10,10
"Brockton, Parkdale Village, Exhibition Place",10,10,10,10,10,10
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",10,10,10,10,10,10
Central Bay Street,10,10,10,10,10,10
Christie,10,10,10,10,10,10
Church and Wellesley,10,10,10,10,10,10
"Commerce Court, Victoria Hotel",10,10,10,10,10,10
Davisville,10,10,10,10,10,10
Davisville North,9,9,9,9,9,9


In [23]:
# Number of unique categories
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 123 uniques categories.


# Analyzing Each Neighborhood

In [24]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arts & Crafts Store,...,Swim School,Tailor Shop,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
toronto_onehot.shape

(344, 123)

### Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [26]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Swim School,Tailor Shop,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.1,0.0,0.0,0.00,0.1,0.0,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.1,0.1,0.1,0.2,0.1,0.1,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0
3,Central Bay Street,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0
4,Christie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0
5,Church and Wellesley,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.1,0.0,0.1,0.00,0.0,0.0,0.0
6,"Commerce Court, Victoria Hotel",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0
7,Davisville,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0
8,Davisville North,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0
9,"Dufferin, Dovercourt Village",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0


### Printing each neighborhood along with the top 5 most common venues

In [27]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
               venue  freq
0     Farmers Market   0.1
1  French Restaurant   0.1
2             Museum   0.1
3       Cocktail Bar   0.1
4       Liquor Store   0.1


----Brockton, Parkdale Village, Exhibition Place----
                venue  freq
0         Coffee Shop   0.2
1           Pet Store   0.1
2  Italian Restaurant   0.1
3                 Bar   0.1
4              Bakery   0.1


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0      Airport Lounge   0.2
1     Harbor / Marina   0.1
2  Airport Food Court   0.1
3        Airport Gate   0.1
4     Airport Service   0.1


----Central Bay Street----
                        venue  freq
0                 Coffee Shop   0.4
1          Italian Restaurant   0.1
2                   Gastropub   0.1
3  Modern European Restaurant   0.1
4   Middle Eastern Restaurant   0.1


----Christie----
                venue  freq
0       Grocery St

In [28]:
# function to sort venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Displaying top 10 venues in each Neighborhood

In [29]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Farmers Market,French Restaurant,Museum,Cocktail Bar,Liquor Store,Restaurant,Beer Bar,Park,Thai Restaurant,Vegetarian / Vegan Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Pet Store,Italian Restaurant,Bar,Bakery,Café,Breakfast Spot,Furniture / Home Store,Gym,Music Venue
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Harbor / Marina,Airport Food Court,Airport Gate,Airport Service,Airport Terminal,Airport,Coffee Shop,Rental Car Location,Modern European Restaurant
3,Central Bay Street,Coffee Shop,Italian Restaurant,Gastropub,Modern European Restaurant,Middle Eastern Restaurant,Japanese Restaurant,Spa,Harbor / Marina,Lake,Gift Shop
4,Christie,Grocery Store,Café,Coffee Shop,Restaurant,Italian Restaurant,Candy Store,Lawyer,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant


# Clustering Neighborhoods

In [30]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 1, 1, 3, 0, 3, 1, 1, 3])

In [31]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Historic Site,Pub,Restaurant,Distribution Center,Breakfast Spot,Park,Bakery,Spa,Gym / Fitness Center
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,3,Café,Burrito Place,Comic Shop,Pizza Place,Music Venue,Plaza,Theater,Clothing Store,Burger Joint,Yoga Studio
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,BBQ Joint,Italian Restaurant,Japanese Restaurant,Middle Eastern Restaurant,Creperie,Cosmetics Shop,Restaurant,Gastropub,Tailor Shop
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Trail,Pub,Health Food Store,Yoga Studio,Italian Restaurant,Museum,Movie Theater,Modern European Restaurant,Middle Eastern Restaurant,Mexican Restaurant
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Farmers Market,French Restaurant,Museum,Cocktail Bar,Liquor Store,Restaurant,Beer Bar,Park,Thai Restaurant,Vegetarian / Vegan Restaurant


In [32]:
# create map
map_clusters = folium.Map(location=[lat, lng], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examing Clusters
### Cluster 1

In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Downtown Toronto,0,Farmers Market,French Restaurant,Museum,Cocktail Bar,Liquor Store,Restaurant,Beer Bar,Park,Thai Restaurant,Vegetarian / Vegan Restaurant
9,East York/East Toronto,0,Pizza Place,Convenience Store,Intersection,Park,Yoga Studio,Jewelry Store,Music Venue,Museum,Movie Theater,Modern European Restaurant
10,Downtown Toronto,0,Performing Arts Venue,Plaza,Lake,Salad Place,Supermarket,Hotel,Dessert Shop,Skating Rink,Park,Liquor Store
12,East Toronto,0,Italian Restaurant,Ice Cream Shop,Greek Restaurant,Cosmetics Shop,Brewery,Yoga Studio,Fruit & Vegetable Store,Gym / Fitness Center,Lake,Music Venue
18,Central Toronto,0,Bus Line,Swim School,Lake,Dim Sum Restaurant,Park,Modern European Restaurant,Middle Eastern Restaurant,Mexican Restaurant,Japanese Restaurant,Liquor Store
21,Central Toronto,0,Trail,Mexican Restaurant,Sushi Restaurant,Jewelry Store,Yoga Studio,Intersection,Museum,Movie Theater,Modern European Restaurant,Middle Eastern Restaurant
22,West Toronto,0,Park,Thai Restaurant,Speakeasy,Bar,Flea Market,Arts & Crafts Store,Antique Shop,Italian Restaurant,Furniture / Home Store,Gastropub
27,Downtown Toronto,0,Yoga Studio,Sushi Restaurant,College Gym,Dessert Shop,Restaurant,Japanese Restaurant,Beer Bar,Bar,Italian Restaurant,French Restaurant
34,Downtown Toronto Stn A,0,Museum,Thai Restaurant,Restaurant,Beer Bar,Fountain,Tailor Shop,French Restaurant,Cocktail Bar,Vegetarian / Vegan Restaurant,Park
37,Downtown Toronto,0,Restaurant,Thai Restaurant,Bookstore,Park,Beer Bar,Mexican Restaurant,Bubble Tea Shop,Breakfast Spot,Theme Restaurant,Dance Studio


### Cluster 2

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Coffee Shop,Historic Site,Pub,Restaurant,Distribution Center,Breakfast Spot,Park,Bakery,Spa,Gym / Fitness Center
2,Downtown Toronto,1,Coffee Shop,BBQ Joint,Italian Restaurant,Japanese Restaurant,Middle Eastern Restaurant,Creperie,Cosmetics Shop,Restaurant,Gastropub,Tailor Shop
3,East Toronto,1,Trail,Pub,Health Food Store,Yoga Studio,Italian Restaurant,Museum,Movie Theater,Modern European Restaurant,Middle Eastern Restaurant,Mexican Restaurant
5,Downtown Toronto,1,Coffee Shop,Italian Restaurant,Gastropub,Modern European Restaurant,Middle Eastern Restaurant,Japanese Restaurant,Spa,Harbor / Marina,Lake,Gift Shop
11,West Toronto,1,Wine Bar,Asian Restaurant,Ice Cream Shop,Korean Restaurant,Cuban Restaurant,Pizza Place,Brewery,Beer Store,Bar,Greek Restaurant
15,East Toronto,1,Park,Fish & Chips Shop,Italian Restaurant,Gym,Ice Cream Shop,Fast Food Restaurant,Pub,Brewery,Liquor Store,Sushi Restaurant
20,Central Toronto,1,Park,Sandwich Place,Gym,Gym / Fitness Center,Playground,Food & Drink Shop,Breakfast Spot,Department Store,Hotel,Mexican Restaurant
23,Central Toronto,1,Yoga Studio,Sporting Goods Shop,Diner,Mexican Restaurant,Clothing Store,Chinese Restaurant,Restaurant,Salon / Barbershop,Spa,Coffee Shop
25,West Toronto,1,Gift Shop,Italian Restaurant,Movie Theater,Cuban Restaurant,Restaurant,Dessert Shop,Dog Run,Coffee Shop,Eastern European Restaurant,Hotel
26,Central Toronto,1,Dessert Shop,Coffee Shop,Sushi Restaurant,Pizza Place,Café,Sandwich Place,Seafood Restaurant,Indian Restaurant,Park,History Museum


### Cluster 3

In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,2,Garden,Park,Music Venue,Museum,Movie Theater,Modern European Restaurant,Middle Eastern Restaurant,Mexican Restaurant,Liquor Store,Lawyer


### Cluster 4

In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,3,Café,Burrito Place,Comic Shop,Pizza Place,Music Venue,Plaza,Theater,Clothing Store,Burger Joint,Yoga Studio
6,Downtown Toronto,3,Grocery Store,Café,Coffee Shop,Restaurant,Italian Restaurant,Candy Store,Lawyer,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant
7,Downtown Toronto,3,Concert Hall,Pizza Place,Restaurant,Speakeasy,Steakhouse,Hotel,Café,Plaza,Vegetarian / Vegan Restaurant,Mexican Restaurant
8,West Toronto,3,Bakery,Grocery Store,Gym / Fitness Center,Music Venue,Bar,Bank,Brewery,Café,Middle Eastern Restaurant,Yoga Studio
13,Downtown Toronto,3,Café,Coffee Shop,Hotel,Pub,Restaurant,Bakery,Japanese Restaurant,Gym / Fitness Center,Gym,Health Food Store
14,West Toronto,3,Coffee Shop,Pet Store,Italian Restaurant,Bar,Bakery,Café,Breakfast Spot,Furniture / Home Store,Gym,Music Venue
16,Downtown Toronto,3,Café,Coffee Shop,Museum,Bakery,Restaurant,American Restaurant,Pub,Gym / Fitness Center,Gym,Movie Theater
17,East Toronto,3,Coffee Shop,Fish Market,Pet Store,Gay Bar,Seafood Restaurant,Café,Bookstore,Ice Cream Shop,Lake,Lawyer
24,Central Toronto,3,Café,Coffee Shop,BBQ Joint,Park,Burger Joint,History Museum,Donut Shop,Middle Eastern Restaurant,Indian Restaurant,Lawyer
28,West Toronto,3,Sushi Restaurant,Falafel Restaurant,Pub,Italian Restaurant,Fish & Chips Shop,Burrito Place,Café,Coffee Shop,Bookstore,Modern European Restaurant


### Cluster 5

In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,Central Toronto,4,Tennis Court,Lawyer,Park,Yoga Studio,Italian Restaurant,Music Venue,Museum,Movie Theater,Modern European Restaurant,Middle Eastern Restaurant
33,Downtown Toronto,4,Park,Trail,Playground,Italian Restaurant,Music Venue,Museum,Movie Theater,Modern European Restaurant,Middle Eastern Restaurant,Mexican Restaurant
